In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

'Colab Notebooks'  'Getting started.pdf'   image   out.csv


# Install Turi Create

In [0]:
!pip install turicreate

     |████████████████████████████████| 91.9MB 32kB/s 
     |████████████████████████████████| 86.3MB 88kB/s 
     |████████████████████████████████| 3.4MB 47.8MB/s 
     |████████████████████████████████| 327kB 61.3MB/s 
     |████████████████████████████████| 3.8MB 42.4MB/s 
     |████████████████████████████████| 450kB 54.8MB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320850 sha256=9652d00c7fbe999d472fc126c5a59f3d847c537efd1a8f17394b08075a5da3d5
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=45a5568cfef82d8bfff446f765f23a1c5763a23a1c15e76301ada44d87536432
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built resampy gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Fo

## Import the .CSV file and format the dataset to create an sframe folder that will be user during training

In [0]:
import turicreate as tc
import os

IMAGES_DIR = '/content/drive/My Drive/image' # Change if applicable
csv_path = '/content/drive/My Drive/out.csv' # assumes CSV column format is image,id,name,xMin,xMax,yMin,yMax
csv_sf = tc.SFrame.read_csv(csv_path)

def row_to_bbox_coordinates(row):
    """
    Takes a row and returns a dictionary representing bounding
    box coordinates:  (center_x, center_y, width, height)  e.g. {'x': 100, 'y': 120, 'width': 80, 'height': 120}
    """
    return {'x': row['xMin'] + (row['xMax'] - row['xMin'])/2,
            'width': (row['xMax'] - row['xMin']),
            'y': row['yMin'] + (row['yMax'] - row['yMin'])/2,
            'height': (row['yMax'] - row['yMin'])}

csv_sf['coordinates'] = csv_sf.apply(row_to_bbox_coordinates)
# delete no longer needed columns
del csv_sf['id'], csv_sf['xMin'], csv_sf['xMax'], csv_sf['yMin'], csv_sf['yMax']
# rename columns
csv_sf = csv_sf.rename({'name': 'label', 'image': 'name'})

# Load all images in random order
sf_images = tc.image_analysis.load_images(IMAGES_DIR, recursive=True,
    random_order=True)

# Split path to get filename
info = sf_images['path'].apply(lambda path: os.path.basename(path).split('/')[:1])

# Rename columns to 'name'
info = info.unpack().rename({'X.0': 'name'})

# Add to our main SFrame
sf_images = sf_images.add_columns(info)

# Original path no longer needed
del sf_images['path']

# Combine label and coordinates into a bounding box dictionary
csv_sf = csv_sf.pack_columns(['label', 'coordinates'], new_column_name='bbox', dtype=dict)

# Combine bounding boxes of the same 'name' into lists
sf_annotations = csv_sf.groupby('name',
    {'annotations': tc.aggregate.CONCAT('bbox')})

# Join annotations with the images. Note, some images do not have annotations,
# but we still want to keep them in the dataset. This is why it is important to
# a LEFT join.
sf = sf_images.join(sf_annotations, on='name', how='left')

# The LEFT join fills missing matches with None, so we replace these with empty
# lists instead using fillna.
sf['annotations'] = sf['annotations'].fillna([])

# Save SFrame
sf.save('/content/plate.sframe')

Finished parsing file /content/drive/My Drive/out.csv

Parsing completed. Parsed 61 lines in 0.01566 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/drive/My Drive/out.csv

Parsing completed. Parsed 61 lines in 0.013076 secs.

# Train the model

In [0]:
import turicreate as tc

tc.config.set_num_gpus(-1)

# Load the data
data =  tc.SFrame('plate.sframe')

# Make a train-test split
train_data, test_data = data.random_split(0.8)

# Create a model
model = tc.object_detector.create(train_data)

# Save predictions to an SArray
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test_data)

# Save the model for later use in Turi Create
model.save('model_plate_turi.model')

# Export for use in Core ML
model.export_coreml('model_plate_turi.mlmodel')

TuriCreate currently only supports using one GPU. Setting 'num_gpus' to 1.
Using 'image' as feature column
Using 'annotations' as annotations column
Download completed: /var/tmp/model_cache/darknet.params
Download completed: /var/tmp/model_cache/darknet.mlmodel


Using CPU to create model.

Setting 'batch_size' to 32

Instructions for updating:
non-resource variables are not supported in the long term


Setting 'max_iterations' to 1000

...

| 997          | 0.756358     | 9h 1m        |

| 998          | 0.736282     | 9h 2m        |

| 999          | 0.743665     | 9h 2m        |

| 1000         | 0.750628     | 9h 2m        |

+--------------+--------------+--------------+

# Download the .mlmodel file 

In [0]:
from google.colab import files
files.download('/content/model_plate_turi.mlmodel')

In [0]:
!zip -r "/content/model_plate_turi_model.zip" "/content/model_plate_turi.model"  